In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q mediapipe==0.10.14


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


## Extracting that rar to colab

In [ ]:
from google.colab import drive
import os

print("="*70)
print("EXTRACTING RAR FROM DRIVE TO COLAB")
print("="*70)



# ===== STEP 2: INSTALL UNRAR =====
print("\n2. Installing unrar tool...")
!apt-get install -y unrar > /dev/null 2>&1
print("✓ unrar installed!")

# ===== STEP 3: SET PATHS =====
# Change this to YOUR RAR file location in Drive
RAR_FILE = "/content/drive/MyDrive/New_Data.rar"  # ← CHANGE THIS

# Extract to Colab disk (much faster than Drive)
EXTRACT_TO = "/content/extracted_videos"

print(f"\n3. Checking RAR file...")
print(f"   RAR location: {RAR_FILE}")

if not os.path.exists(RAR_FILE):
    print("RAR file not found!")
    print("   Please update RAR_FILE path above")
else:
    print("✓ RAR file found!")

    # Get file size
    size_mb = os.path.getsize(RAR_FILE) / (1024*1024)
    print(f"   Size: {size_mb:.1f} MB")

# ===== STEP 4: EXTRACT =====
if os.path.exists(RAR_FILE):
    print(f"\n4. Extracting to {EXTRACT_TO}...")
    print("   This may take a few minutes...")

    # Create extraction directory
    os.makedirs(EXTRACT_TO, exist_ok=True)

    # Extract RAR
    !unrar x "{RAR_FILE}" "{EXTRACT_TO}/"

    print("\n✓ Extraction complete!")

    # ===== STEP 5: VERIFY =====
    print("\n5. Verifying extracted files...")

    # Count video files
    video_count = 0
    for root, dirs, files in os.walk(EXTRACT_TO):
        for file in files:
            if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.webm')):
                video_count += 1

    print(f"✓ Found {video_count} video files")

    # Show directory structure
    print(f"\nExtracted structure:")
    !ls -lh "{EXTRACT_TO}"

    print("\n" + "="*70)
    print("READY! Videos extracted to Colab disk")
    print("="*70)
    print(f"\nYour videos are at: {EXTRACT_TO}")
    print(f"   Use this path in your code!")
else:
    print("\nCannot proceed without RAR file")
    print("   Please upload your RAR to Google Drive first")


📦 EXTRACTING RAR FROM DRIVE TO COLAB

2. Installing unrar tool...
✓ unrar installed!

3. Checking RAR file...
   RAR location: /content/drive/MyDrive/New_Data.rar
✓ RAR file found!
   Size: 52.1 MB

4. Extracting to /content/extracted_videos...
   This may take a few minutes...

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/New_Data.rar

Creating    /content/extracted_videos/New_Data                        OK
Extracting  /content/extracted_videos/New_Data/i1.mp4                      2%  OK 
Extracting  /content/extracted_videos/New_Data/i10.mp4                     2%  OK 
Extracting  /content/extracted_videos/New_Data/i11.mp4                     3%  OK 
Extracting  /content/extracted_videos/New_Data/i12.mp4                     4%  OK 
Extracting  /content/extracted_videos/New_Data/i13.mp4                     5%  OK 
Extracting  /content/extracted_videos/New_Data/i14.mp4     

## Extracting Keypoints without Preprocessing

In [8]:
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from mediapipe.python.solutions import holistic as mp_holistic

print("="*70)
print("ULTRA FAST: VIDEO → NUMPY (NO PREPROCESSING)")
print("="*70)

VIDEO_DIR = "/content/extracted_videos/New_Data"
NPY_DIR = "/content/drive/MyDrive/FYP_word/my_data_array"
CSV_PATH = '/content/drive/MyDrive/FYP_word/My_data.csv'

os.makedirs(NPY_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)
print(f"Videos to process: {len(df)}")

holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.3
)

success = 0
failed = 0

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting"):
    video_id = row['video_id']
    video_path = f"{VIDEO_DIR}/{video_id}.mp4"
    npy_path = f"{NPY_DIR}/{video_id}.npy"

    if os.path.exists(npy_path):
        success += 1
        continue

    if not os.path.exists(video_path):
        failed += 1
        continue

    try:
        cap = cv2.VideoCapture(video_path)
        frames_data = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # SIMPLE: Just convert to RGB, NO denoising/enhancement
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)

            pose_vector = []

            if results.pose_landmarks:
                for lm in results.pose_landmarks.landmark:
                    pose_vector.extend([lm.x, lm.y, lm.z])
            else:
                pose_vector.extend([0.0] * 99)

            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    pose_vector.extend([lm.x, lm.y, lm.z])
            else:
                pose_vector.extend([0.0] * 63)

            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    pose_vector.extend([lm.x, lm.y, lm.z])
            else:
                pose_vector.extend([0.0] * 63)

            frames_data.append(pose_vector)

        cap.release()

        if len(frames_data) >= 5:
            pose_array = np.array(frames_data, dtype=np.float32)
            np.save(npy_path, pose_array)
            success += 1
        else:
            failed += 1

    except:
        failed += 1

holistic.close()

print(f"\n{'='*70}")
print(f"✓ Success: {success}/{len(df)}")
print(f"✗ Failed: {failed}")

successful_ids = [f.replace('.npy', '') for f in os.listdir(NPY_DIR) if f.endswith('.npy')]
df_ready = df[df['video_id'].isin(successful_ids)]

TRAIN_CSV = '/content/drive/MyDrive/FYP_word/new_my_cv.csv'
df_ready.to_csv(TRAIN_CSV, index=False)



ULTRA FAST: VIDEO → NUMPY (NO PREPROCESSING)
Videos to process: 75


Extracting:   0%|          | 0/75 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Extracting: 100%|██████████| 75/75 [16:58<00:00, 13.58s/it]


✓ Success: 75/75
✗ Failed: 0


In [13]:
import os
import pandas as pd


NPY_DIR = "/content/drive/MyDrive/FYP_word/top20_npy_arrays_with_my_data"
CSV_PATH = '/content/drive/MyDrive/FYP_word/top15_with_my_data.csv'


npy_files = [f for f in os.listdir(NPY_DIR) if f.endswith('.npy')]
print(f"Found {len(npy_files)} existing .npy files!")


df = pd.read_csv(CSV_PATH)
# df['video_id'] = df['video_id'].astype(str).str.zfill(5)
df['video_id'] = df['video_id'].astype(str)  # No .zfill(5)



# Match with existing .npy files
existing_ids = set([f.replace('.npy', '') for f in npy_files])
df_ready = df[df['video_id'].isin(existing_ids)]

print(f"\n{'='*70}")
print(f"MATCHED DATASET")
print(f"{'='*70}")
print(f"Total matched: {len(df_ready)}")
print(f"Classes: {df_ready['gloss'].nunique()}")

if len(df_ready) > 0:
    print(f"\nPer-word breakdown:")
    print(df_ready['gloss'].value_counts().sort_index())

    # Save training CSV
    TRAIN_CSV = '/content/drive/MyDrive/FYP_word/new_my_cv.csv'
    df_ready.to_csv(TRAIN_CSV, index=False)

    print(f"\n✅ READY TO TRAIN!")
    print(f"CSV: {TRAIN_CSV}")
    print(f"NPY folder: {NPY_DIR}")
    print(f"Videos: {len(df_ready)}")
    print(f"Avg per class: {len(df_ready)/df_ready['gloss'].nunique():.1f}")
else:
    print("\nNo matches - video IDs don't align")


Found 266 existing .npy files!

MATCHED DATASET
Total matched: 225
Classes: 16

Per-word breakdown:
gloss
computer    14
cousin      14
deaf        11
drink       13
go          15
help        13
i           25
like        10
no          11
play        27
please      23
thin        13
walk        10
who         10
year         8
yes          8
Name: count, dtype: int64

✅ READY TO TRAIN!
CSV: /content/drive/MyDrive/FYP_word/new_my_cv.csv
NPY folder: /content/drive/MyDrive/FYP_word/top20_npy_arrays_with_my_data
Videos: 225
Avg per class: 14.1


## Reducing Dataset to 10+ samples per word

In [ ]:
import pandas as pd

CSV_PATH = '/content/drive/MyDrive/FYP_word/top20_bilstm_ready.csv'
df = pd.read_csv(CSV_PATH)

# Keep only words with 10+ samples
word_counts = df['gloss'].value_counts()
good_words = word_counts[word_counts >= 10].index.tolist()

df_filtered = df[df['gloss'].isin(good_words)]

print(f"{'='*70}")
print(f"FILTERED TO 10+ SAMPLES PER WORD")
print(f"{'='*70}")
print(f"Words: {len(good_words)}")
print(f"Videos: {len(df_filtered)}")
print(f"Avg per word: {len(df_filtered)/len(good_words):.1f}")

print(f"\nFinal dataset:")
print(df_filtered['gloss'].value_counts().sort_index())

# Save
FINAL_CSV = '/content/drive/MyDrive/FYP_word/top15_train_ready.csv'
df_filtered.to_csv(FINAL_CSV, index=False)

print(f"\n✅ SAVED: {FINAL_CSV}")
print(f"\n Expected accuracy: 40-48%")
print(f"Top-5 accuracy: 70-78%")


FILTERED TO 10+ SAMPLES PER WORD
Words: 13
Videos: 167
Avg per word: 12.8

Final dataset:
gloss
computer    14
cousin      14
deaf        11
drink       15
go          15
help        14
like        10
no          11
thin        16
walk        11
who         14
year        10
yes         12
Name: count, dtype: int64

✅ SAVED: /content/drive/MyDrive/FYP_word/top15_train_ready.csv

🎯 Expected accuracy: 40-48%
🎯 Top-5 accuracy: 70-78%
